# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import googlemaps

# Import API key
from api_keys import g_key

### Use csv from Weather Py 

In [2]:
weather_file = pd.read_csv("../output_data/city.csv")
weather_file.dropna()
weather_file


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Melbourne,1,US,1619905204,57,28.0836,-80.6081,82.40,11.50
1,1,Tuktoyaktuk,90,CA,1619904796,92,69.4541,-133.0374,14.00,5.75
2,2,Olinda,75,BR,1619905230,69,-8.0089,-34.8553,80.60,9.22
3,3,Griffith,1,US,1619905230,36,41.5284,-87.4237,80.60,19.57
4,4,Grand-Santi,73,GF,1619905231,97,4.2500,-54.3833,74.35,2.68
...,...,...,...,...,...,...,...,...,...,...
121,121,Carnarvon,100,AU,1619905106,64,-24.8667,113.6333,68.00,10.36
122,122,Anadyr,40,RU,1619905263,79,64.7500,177.4833,14.00,11.18
123,123,Ilulissat,18,GL,1619905264,42,69.2167,-51.1000,46.40,12.66
124,124,Kwinana,0,AU,1619905264,100,-32.2500,115.7667,48.20,5.75


### Humidity Heatmap


In [3]:
#Load gmaps gkey
gmaps.configure(api_key=g_key)

In [4]:

locations = weather_file[["Lat", "Lng"]]
humidity = weather_file["Humidity"].astype(float)

In [5]:
hfigure = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
hfigure.add_layer(heat_layer)
#Display figure
hfigure

Figure(layout=FigureLayout(height='420px'))

### Cities meeting ideal weather conditions

In [6]:
temp_file=weather_file.loc[(weather_file['Max Temp']>=60)&(weather_file['Max Temp']<=90)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']<=10)]
ideal_weather



,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
18,18,Chui,1,UY,1619905235,90,-33.6971,-53.4616,63.27,6.13
25,25,Cabo San Lucas,1,MX,1619905105,65,22.8909,-109.9124,84.99,8.05
27,27,Puerto Maldonado,0,PE,1619905238,54,-12.6000,-69.1833,86.00,3.44
33,33,Faya,0,SA,1619905180,24,18.3851,42.4509,68.00,3.44
34,34,Cumberland,1,US,1619905239,13,35.0502,-78.8664,77.00,5.75
56,56,Russell,1,US,1619905246,26,32.3502,-85.1999,82.40,2.95
61,61,Mount Isa,2,AU,1619905247,55,-20.7333,139.5000,66.20,4.61
65,65,Port Alfred,0,ZA,1619905122,65,-33.5906,26.8910,61.09,5.28
70,70,Henties Bay,0,NaN,1619905167,90,-22.1160,14.2845,61.00,2.10
81,81,Atuona,2,PF,1619905064,69,-9.8000,-139.0333,79.03,6.17


### Hotel Map

In [7]:
possible_hotels = ideal_weather.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
possible_hotels["Hotel Name"] = ""
possible_hotels

,City,Country,Lat,Lng,Hotel Name
18,Chui,UY,-33.6971,-53.4616,
25,Cabo San Lucas,MX,22.8909,-109.9124,
27,Puerto Maldonado,PE,-12.6000,-69.1833,
33,Faya,SA,18.3851,42.4509,
34,Cumberland,US,35.0502,-78.8664,
56,Russell,US,32.3502,-85.1999,
61,Mount Isa,AU,-20.7333,139.5000,
65,Port Alfred,ZA,-33.5906,26.8910,
70,Henties Bay,NaN,-22.1160,14.2845,
81,Atuona,PF,-9.8000,-139.0333,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in possible_hotels.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    #base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    json.dumps(hotel_name, indent=4, sort_keys=True)
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        possible_hotels.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        
possible_hotels

IndentationError: expected an indented block (<ipython-input-9-7866b7e2e7d5>, line 26)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in possible_hotels.iterrows()]
locations = possible_hotels[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

hfigure.add_layer(markers)

# Display Map
hfigure
